Starting off, we needed to import the required dependencies for our project. Importing our libraries and prepping the CSV file to be read and interpretted was the first step. One interesting challenge that we encountered early on was the need to break out the dates provided into a useable format for our needs. This involved breaking out the date field that was originally delivered in a YYYY-MM-DD format into three separate columns themselves. 

In [5]:
#import dependencies
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#read csv
csv_path = "../Resources/big_five_stocks.csv"
stock_df = pd.read_csv(csv_path, encoding="utf-8")
#rename the date column to a more useable name
astock = stock_df.rename( columns={'Unnamed: 0':'date'}, inplace=False )
#format the date column to be recognized as a datetime object
astock['Formatted Date'] = pd.to_datetime(astock['date'])
#isolate Year, Month, and Day into separate columns
astock['Month'] = astock['Formatted Date'].map(lambda x: x.month)
astock['Year'] = astock['Formatted Date'].map(lambda x: x.year)
astock['Day'] = astock['Formatted Date'].map(lambda x: x.day)
#Filter down to last 10 years
filterstock = astock[(astock['Year'] >= 2009) & (astock['Year'] <= 2019)]
filterstock.head()

,date,name,open,close,high,low,volume,Formatted Date,Month,Year,Day
26437,2009-01-02,AAPL,12.23,12.96,13.01,12.17,26964210.0,2009-01-02,1,2009,2
26438,2009-01-02,AMZN,51.10,54.36,54.53,51.07,7296667.0,2009-01-02,1,2009,2
26439,2009-01-02,GOOGL,154.01,160.66,160.91,152.75,3617574.0,2009-01-02,1,2009,2
26440,2009-01-02,^IXIC,1578.87,1632.21,1635.83,1572.04,0.0,2009-01-02,1,2009,2
26441,2009-01-02,MSFT,19.54,20.33,20.40,19.37,50121848.0,2009-01-02,1,2009,2


Now that we had the data in the notebook, we needed to isolate the important columns and sort the information appropriately. We decided that we only needed the date related fields we would sort by and the accompanying stock data.

In [6]:
#isolate only useful information
organizedstock = filterstock[['name','Year','Month', 'Day', 'open','close','high','low', 'volume']]
#sort by name and date to organize our stock data chronologically
sortedstock = organizedstock.sort_values(by=['name', 'Year', 'Month', 'Day'])
sortedstock

,name,Year,Month,Day,open,close,high,low,volume
26437,AAPL,2009,1,2,12.23,12.96,13.01,12.17,26964210.0
26446,AAPL,2009,1,5,13.30,13.51,13.74,13.24,42458780.0
26448,AAPL,2009,1,6,13.71,13.29,13.88,13.20,46149128.0
26452,AAPL,2009,1,7,13.09,13.00,13.21,12.89,27042958.0
26458,AAPL,2009,1,8,12.92,13.24,13.31,12.86,24052284.0
...,...,...,...,...,...,...,...,...,...
41630,^IXIC,2019,8,19,8006.18,8002.81,8026.75,7974.36,0.0
41636,^IXIC,2019,8,20,7989.36,7948.56,8010.58,7948.09,0.0
41645,^IXIC,2019,8,21,8017.07,8020.21,8036.94,7998.50,0.0
41651,^IXIC,2019,8,22,8038.79,7991.39,8048.58,7937.13,0.0


The first of the values we decided to gather was the total amount of stock sold by each company over the 10 year period. This was done with a simple for loop in which we used the loc and groupby functions to isolate the volumes and sum them by month.

In [4]:
#For loop to get total sum of vol by month
months = [1,2,3,4,5,6,7,8,9,10,11,12]
for month in months:
    totalvolbymo = sortedstock.loc[sortedstock['Month'] == month].groupby('name')['volume'].sum()
    print(totalvolbymo)

name
AAPL     7.236981e+09
AMZN     1.359727e+09
FB       5.719159e+09
GOOGL    6.494569e+08
MSFT     1.134852e+10
^IXIC    0.000000e+00
Name: volume, dtype: float64
name
AAPL     6.085345e+09
AMZN     1.276771e+09
FB       4.626707e+09
GOOGL    5.584776e+08
MSFT     9.235426e+09
^IXIC    0.000000e+00
Name: volume, dtype: float64
name
AAPL     6.244181e+09
AMZN     1.189071e+09
FB       4.790840e+09
GOOGL    5.962936e+08
MSFT     9.900353e+09
^IXIC    0.000000e+00
Name: volume, dtype: float64
name
AAPL     5.787338e+09
AMZN     1.243370e+09
FB       5.316049e+09
GOOGL    6.113509e+08
MSFT     1.045622e+10
^IXIC    0.000000e+00
Name: volume, dtype: float64
name
AAPL     6.039379e+09
AMZN     1.004306e+09
FB       5.335489e+09
GOOGL    5.096403e+08
MSFT     9.584783e+09
^IXIC    0.000000e+00
Name: volume, dtype: float64
name
AAPL     6.149316e+09
AMZN     9.685126e+08
FB       4.537416e+09
GOOGL    5.212479e+08
MSFT     1.008460e+10
^IXIC    0.000000e+00
Name: volume, dtype: float64
name

The next set of values we needed to isolate were the opening and closing values for each month across each year for each company over the 10 year period. We spent a good amount of time trying to figure out how we were going to approach this. Instead of opting for a nest for loop, we decided to isolate the opening and closing dates by manipulating the original data frame and merging the manipulate dataframe with the original to isolate the values we were after. 

In [7]:
#group by name year and month
groupedstock = sortedstock.groupby(['name','Year','Month'])
#isolate the lowest day available for each month by year and name
firstday = groupedstock['Day'].min()
#isolate the highest day available for each month by year and name
lastday = groupedstock['Day'].max()

In [10]:
#create data frame to house the above values
#create the initial dataframe off the firstday values
summary = pd.DataFrame(firstday)
#add a column to house the lastday values
summary['Last Day'] = lastday
summary.reset_index()

,name,Year,Month,Day,Last Day
0,AAPL,2009,1,2,30
1,AAPL,2009,2,2,27
2,AAPL,2009,3,2,31
3,AAPL,2009,4,1,30
4,AAPL,2009,5,1,29
...,...,...,...,...,...
723,^IXIC,2019,4,1,30
724,^IXIC,2019,5,1,31
725,^IXIC,2019,6,3,28
726,^IXIC,2019,7,1,31


In [11]:
#merge the above dataframe to the original on the opening date to isolate the opening date values for each close date of each month by year and name
mergeddf = pd.merge(summary, sortedstock, on=['name','Year','Month','Day'])
#reorganizing data for readibility
cleandf = mergeddf[['name','Year','Month','Day','Last Day','open']]
#merge the newly create dataframe with the original on the closing date to isolate the closing date values for each close date of each month by year and name
twomerge = pd.merge(cleandf, sortedstock, left_on=["Last Day", 'name','Year','Month'], right_on=["Day", 'name','Year','Month'])
#Reorganizing column names for readibility
cleanmerge = twomerge[['name','Year','Month','Day_x','Last Day','open_x','close']]
#cleaning column names
openclosedf = cleanmerge.rename( columns={'name':'Name', 'Day_x':'First Day', 'open_x':'Open'}, inplace=False )
openclosedf

,Name,Year,Month,First Day,Last Day,Open,close
0,AAPL,2009,1,2,30,12.23,12.88
1,AAPL,2009,2,2,27,12.71,12.76
2,AAPL,2009,3,2,31,12.57,15.02
3,AAPL,2009,4,1,30,14.86,17.98
4,AAPL,2009,5,1,29,17.97,19.40
...,...,...,...,...,...,...,...
723,^IXIC,2019,4,1,30,7800.24,8095.39
724,^IXIC,2019,5,1,31,8132.93,7453.15
725,^IXIC,2019,6,3,28,7441.22,8006.24
726,^IXIC,2019,7,1,31,8145.85,8175.42


## Plotting